In [1]:
!git clone https://github.com/harshavardhan-patil/mimolm.git

Cloning into 'mimolm'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 275 (delta 139), reused 209 (delta 77), pack-reused 0 (from 0)
Receiving objects: 100% (275/275), 340.44 KiB | 37.83 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [2]:
%cd mimolm
!git checkout feature/lambda

/home/ubuntu/mimolm
Branch 'feature/lambda' set up to track remote branch 'feature/lambda' from 'origin'.
Switched to a new branch 'feature/lambda'


In [3]:
%pip install lightning
%pip install python-dotenv
%pip install loguru
%pip install transforms3d
%pip install omegaconf
%pip install tensordict
%pip install torch-kmeans

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 KB 10.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 KB 19.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 KB 26.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.6/126.6 KB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.3/315.3 KB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 KB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 61.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need 

In [2]:
import sys
from pathlib import Path
from os.path  import join

# Append the directory to your python path using sys
sys.path.append('/home/ubuntu/mimolm')
# Add the project root to sys.path
print(Path.cwd()) 

/home/ubuntu


In [3]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2
from src.mimolm import MimoLM
import lightning as pl
from lightning.pytorch.tuner import Tuner
import time
import torch
import tqdm
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor

data_module = DataH5av2("/home/ubuntu/mimolm/data")
data_module.setup(stage="fit")
train_loader = data_module.train_dataloader()
#val_loader = data_module.val_dataloader()


model = MimoLM(data_size=data_module.tensor_size_train
               , n_rollouts = 1
               , learning_rate = 1.e-04,)

checkpoint_callback = ModelCheckpoint(
    save_top_k=-1,  # Save all checkpoints
    every_n_epochs=1  # Save checkpoint every n epochs
)
lr_monitor = LearningRateMonitor(logging_interval='epoch')
trainer = pl.Trainer(precision='16-mixed',
                     callbacks=[checkpoint_callback, lr_monitor],
                     max_epochs=1,
                     profiler="simple",)
                     #default_root_dir="/content/drive/MyDrive/Colab/mimolm/ckpts")
# tuner = Tuner(trainer)

# #Run learning rate finder and then train
# lr_finder = tuner.lr_find(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)
trainer.fit(model=model, train_dataloaders=train_loader), #val_dataloaders=val_loader)#, ckpt_path='/content/drive/MyDrive/Colab/mimolm/ckpts/lightning_logs/version_1/checkpoints/epoch=4-step=20825.ckpt')


2025-02-11 01:01:24.116 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/ubuntu/mimolm
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ubuntu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GH200 480GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-02-11 01:01:26.808557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already

Epoch 0:   0%|          | 0/196 [00:00<?, ?it/s] 

/home/ubuntu/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 128. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 0: 100%|██████████| 196/196 [01:53<00:00,  1.73it/s, v_num=1, train_loss=1.190]

/home/ubuntu/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 28. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 196/196 [01:53<00:00,  1.73it/s, v_num=1, train_loss=1.190]


FIT Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                        	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                         	|  -              	|  8284           	|  

(None,)

In [5]:
import multiprocessing

multiprocessing.cpu_count()

64